In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

data = pd.read_csv("boston.csv")
print(data.head())

data.dropna(inplace=True)
scaler = StandardScaler()
X = scaler.fit_transform(data.drop(columns=["MEDV"]))
y = data["MEDV"].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

svm_classic = SVR(kernel='linear')
svm_classic.fit(X_train, y_train)
y_pred_classic = svm_classic.predict(X_test)

svm_rbf = SVR(kernel='rbf')
svm_rbf.fit(X_train, y_train)
y_pred_rbf = svm_rbf.predict(X_test)


plt.figure(figsize=(12, 6))
plt.scatter(y_test, y_pred_classic, color='blue', label='Класичний SVM (Linear)')
plt.scatter(y_test, y_pred_rbf, color='red', label='Проградувальний SVM (RBF)')
plt.xlabel('Реальні значення')
plt.ylabel('Прогнозовані значення')
plt.legend()
plt.title('Порівняння класичної та проградувальної SVM моделей')
plt.show()

plt.figure(figsize=(12, 6))
sns.histplot(y_test - y_pred_rbf, bins=30, kde=True, color='purple')
plt.title("Розподіл залишків (y_test - y_pred_rbf)")
plt.xlabel("Залишки")
plt.ylabel("Частота")
plt.show()

print("Класичний SVM")
print("MSE:", mean_squared_error(y_test, y_pred_classic))
print("MAE:", mean_absolute_error(y_test, y_pred_classic))
print("R2 Score:", r2_score(y_test, y_pred_classic))

print("\nПроградувальний SVM")
print("MSE:", mean_squared_error(y_test, y_pred_rbf))
print("MAE:", mean_absolute_error(y_test, y_pred_rbf))
print("R2 Score:", r2_score(y_test, y_pred_rbf))

param_grid = {
    'C': [0.1, 1, 10, 100],
    'epsilon': [0.01, 0.1, 1, 5],
    'kernel': ['linear', 'rbf']
}
grid_search = GridSearchCV(SVR(), param_grid, cv=5)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("\nОптимальні параметри:", best_params)

best_svm = grid_search.best_estimator_
y_pred_best = best_svm.predict(X_test)

print("\nОцінка оптимізованої моделі")
print("MSE після оптимізації:", mean_squared_error(y_test, y_pred_best))
print("MAE після оптимізації:", mean_absolute_error(y_test, y_pred_best))
print("R2 Score після оптимізації:", r2_score(y_test, y_pred_best))


print("\nЗміни параметрів після оптимізації:")
print(f"C: {best_params['C']}, epsilon: {best_params['epsilon']}, kernel: {best_params['kernel']}")
